In [1]:
import numpy as np
import pandas as pd
import time
import random
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from tqdm import tqdm
import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchmetrics
import tenseal as ts

In [2]:
# Importing, balancing, scaling and splitting the dataset into train and test set
df = pd.read_csv(r"C:\Users\manig\Downloads\Mitacs\top_feature_df_rfr40.csv")

grouped = df.groupby('Label')
df = grouped.apply(lambda x: x.sample(grouped.size().min(), random_state=73).reset_index(drop=True))

x = df.drop(["Label"], axis = 1)
y = df["Label"]
# scaler = preprocessing.MinMaxScaler()
scaler = preprocessing.StandardScaler()
x = scaler.fit_transform(x)

x_train, x_test_20, y_train, y_test_20 = train_test_split(x, y, test_size=0.2, random_state=42)
x_train=torch.from_numpy(x_train).float().unsqueeze(dim=1)
x_test_20=torch.from_numpy(x_test_20).float().unsqueeze(dim=1)
y_train=torch.from_numpy(np.array(y_train)).float().unsqueeze(dim=1)
y_test_20=torch.from_numpy(np.array(y_test_20)).float().unsqueeze(dim=1)
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test_20.shape}")
print(f"y_test shape: {y_test_20.shape}")

x_train shape: torch.Size([681185, 1, 40])
y_train shape: torch.Size([681185, 1])
x_test shape: torch.Size([170297, 1, 40])
y_test shape: torch.Size([170297, 1])


In [3]:
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [21]:
class ONEDCNN(torch.nn.Module):
    def __init__(self):
        super(ONEDCNN, self).__init__()
        self.conv_layer = torch.nn.Conv1d(in_channels=1, out_channels=3, kernel_size=5, stride=1)
        self.dense_layer_1 = torch.nn.Linear(in_features=108, out_features=32)
        self.dense_layer_2 = torch.nn.Linear(in_features=32, out_features=1)
        
    def forward(self, x):
        x = self.conv_layer(x)
        x = x * x
        x = x.view(x.size(0), -1)
        x = self.dense_layer_1(x)
        x = x * x
        x = self.dense_layer_2(x)
        return x

In [18]:
# Function to calculate accuracy, recall, precision and f1 - score
def metrics_fn(y_true, y_pred, others=False):
    for i in range(len(y_pred)):
        if y_pred[i]<0.5:
            y_pred[i] = 0
        else:
            y_pred[i] = 1
            
    accuracy = torchmetrics.Accuracy(task="binary", num_classes=2)
    acc = accuracy(y_pred, y_true)
    
    if others==True:
        rec = torchmetrics.Recall(task="binary", num_classes=2)
        prec = torchmetrics.Precision(task="binary", num_classes=2)
        recall = rec(y_pred, y_true)
        precision = prec(y_pred, y_true)
        f1_score = 2*((precision*recall)/(precision+recall))
        del accuracy, rec, prec
        return acc, recall, precision, f1_score
    
    return acc

In [19]:
# function to train the model
def train(model, train_loader, criterion, optimizer, n_epochs=10):
    loss_dict = {}
    acc_dict = {}
    model.train()
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        train_acc = 0.0
        for data, target in train_loader:
            output = model(data)
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
#             scheduler.step(loss)
            train_loss += loss.item()
            train_acc += metrics_fn(target, torch.sigmoid(output))
        train_loss = train_loss / len(train_loader)
        train_acc = train_acc / len(train_loader)
        loss_dict[epoch] = train_loss
        acc_dict[epoch] = train_acc
        if epoch%3 == 0:
            print(f"Epoch: {epoch} Training Loss: {train_loss} || Training Accuracy: {train_acc} || Learning Rate: {optimizer.param_groups[0]['lr']}")
    model.eval()
    return model, loss_dict, acc_dict

In [23]:
torch.manual_seed(42)
model = ONEDCNN()

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model, loss_dict, acc_dict = train(model, train_loader, criterion, optimizer, 24)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5)) 
x_coord, y_coord = zip(*(loss_dict.items()))
ax1.plot(x_coord, y_coord)
x_coord, y_coord = zip(*(acc_dict.items()))
ax2.plot(x_coord, y_coord)

Epoch: 3 Training Loss: 2.1311208665581285 || Training Accuracy: 0.9430975317955017 || Learning Rate: 0.001


KeyboardInterrupt: 

In [167]:
model.conv_layer.weight.data.view(
            model.conv_layer.out_channels, model.conv_layer.kernel_size[0]
        ).tolist(),model.conv_layer.bias.data.tolist()

([[0.1324390023946762,
   0.1665675789117813,
   -0.16453391313552856,
   0.18222437798976898,
   -0.17997416853904724],
  [0.022764118388295174,
   -0.1675294041633606,
   0.09291396290063858,
   0.18811888992786407,
   -0.2205113172531128],
  [0.21267381310462952,
   -0.14515002071857452,
   0.10244119167327881,
   -0.0516599640250206,
   0.033398158848285675]],
 [-0.5036184787750244, 1.1617426872253418, -0.11102926731109619])

In [168]:
bias = model.conv_layer.bias.data.tolist()

In [189]:
class EncONEDCNN:
    def __init__(self, torch_nn):
        self.conv_layer_weight = torch_nn.conv_layer.weight.data.view(
            torch_nn.conv_layer.out_channels, torch_nn.conv_layer.kernel_size[0]
        ).tolist()
        self.conv_layer_bias = torch_nn.conv_layer.bias.data.tolist()
        
        self.dense_layer_1_weight = torch_nn.dense_layer_1.weight.T.data.tolist()
        self.dense_layer_1_bias = torch_nn.dense_layer_1.bias.data.tolist()
        
        self.dense_layer_2_weight = torch_nn.dense_layer_2.weight.T.data.tolist()
        self.dense_layer_2_bias = torch_nn.dense_layer_2.bias.data.tolist()
        
    def forward(self, enc_x):
        enc_channels = []
        for kernel, bias in zip(self.conv_layer_weight, self.conv_layer_bias):
            y = enc_x.mm(kernel) + bias
            enc_channels.append(y)
        enc_x = ts.CKKSVector.pack_vectors(enc_channels)
        enc_x.square_()
        enc_x = enc_x.mm(self.dense_layer_1_weight) + self.dense_layer_1_bias
        enc_x.squarae_()
        enc_x = enc_x.mm(self.dense_layer_2_weight) + self.dense_layer_2_bias
        return enc_x
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [172]:
def im2row(x):
#     kernel_shape  = kernel.shape[1]
    kernel_shape = 5
    rows = []
    end_neglet = 4 # needs to be changed when the input shape is changed
    # Padding = 0, stride = 1
    for row in range(x.shape[0] - end_neglet):
        window = x[row: row + kernel_shape]
        rows.append(window.flatten())
    return np.vstack(rows)

In [187]:
def enc_test(context, model, test_loader, criterion, kernel_shape, stride, tqdm_batch):
    tar = []
    op = []
    acc = 0.0
    test_loss = 0.0
    for data, target in tqdm(test_loader):
        x_enc = []
        im2row_matrix = im2row(input_matrix) 
        for i in im2row_matrix:
            x_enc.append(ts.ckks_vector(context, i))
#         x_enc = ts.ckks_vector(context, im2row_matrix)
        enc_output = enc_model(x_enc)
        output = enc_output.decrypt()
        print(output)
        output = torch.tensor(output).view(1, -1)
        loss = criterion(output, target)
        test_loss += loss.item()
        output = torch.sigmoid(output)
        for i in range(len(output)):
            if output[i]<0.5:
                output[i] = 0
            else:
                output[i] = 1
        tar.extend(target)
        opt = output.detach()
        op.extend(opt)
        acc += metrics_fn(y_true=target, y_pred=output)
    test_loss = test_loss / len(test_loader)
    print(f'Test Loss: {test_loss}')
    acc /= len(test_loader)
    print(f"Batch {tqdm_batch} Encrypted Accuracy: {acc}")
#     print(f'Accuracy: {acc}')
    return tar, op

In [190]:
enc_model = EncONEDCNN(model)
y_target_final = []
y_pred_final = []
one_part = y_test_20.shape[0] // 20
for i in range(1, 21):
    if i==1:
        j = one_part
        temp_x_test = x_test_20[:j,]
        temp_y_test = y_test_20[:j,]
    elif i==20:
        j = one_part * (i-1)
        temp_x_test = x_test_20[j:,]
        temp_y_test = y_test_20[j:,]
    else:
        j = one_part * i
        if i == 1:
            k = one_part
        else:
            k = one_part * (i-1)
        temp_x_test = x_test_20[k:j,]
        temp_y_test = y_test_20[k:j,]
    
    test_dataset = torch.utils.data.TensorDataset(temp_x_test, temp_y_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)
    
    t_start = time.time()
    enc_x_test = []
    print()
    print(f"Starting Batch {i} Encryption & Evaluation")
    target, output = enc_test(ctx_eval, enc_model, test_loader, criterion, kernel_shape, stride, tqdm_batch=i)
    t_end = time.time()
    print(f"Encryption & Evaluation of the test set took {int(t_end - t_start)} seconds")
    y_pred_final.extend(output)
    y_target_final.extend(target)


Starting Batch 1 Encryption & Evaluation


  0%|                                                                                         | 0/8514 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'mm'

In [36]:
model.conv_layer.weight

Parameter containing:
tensor([[[ 0.1324,  0.1666, -0.1645,  0.1822, -0.1800]],

        [[ 0.0228, -0.1675,  0.0929,  0.1881, -0.2205]],

        [[ 0.2127, -0.1452,  0.1024, -0.0517,  0.0334]]], requires_grad=True)

In [89]:
model.conv_layer.bias

Parameter containing:
tensor([-0.5036,  1.1617, -0.1110], requires_grad=True)

In [154]:
def im2row(x, kernel):
    kernel_shape = kernel.shape[1]
    rows = []
    end_neglet = 4 # needs to be changed when the input shape is changed
    # Padding = 0, stride = 1
    for row in range(x.shape[0] - end_neglet):
        window = x[row: row + kernel_shape]
        rows.append(window.flatten())
    return np.vstack(rows)

In [156]:
output_shape = (input_matrix.shape[0] - kernel.shape[1]) + 1
im2row_matrix = im2row(input_matrix, kernel) 

In [170]:
result = []
for i, j in zip(kernel, bias):
    result.append(np.matmul(im2row_matrix,i) + j)
result
# result = np.array(result)
# result = result.reshape(result[0].shape[0]*kernel.shape[0], 1)
# result = result.tolist()

[array([-0.41859806, -0.5891209 , -0.9327191 , -0.05819717, -1.0278276 ,
        -0.18727016, -0.41709524, -0.73032266, -0.33717138, -0.6717379 ,
        -0.29773414, -0.4164884 , -0.49897432, -0.46913457, -0.47087964,
        -0.5279903 , -0.47726634, -0.43855357, -0.7479829 , -0.36894122,
        -0.6947897 , -0.6359041 , -0.45267066, -0.5518055 , -0.49114344,
        -0.523178  , -0.5392168 , -0.5925485 , -0.3761148 , -0.5365835 ,
        -0.58710253, -0.63822335, -0.6048598 , -0.6683191 , -0.56740606,
        -0.55592036], dtype=float32),
 array([1.16872  , 1.330668 , 0.6467402, 1.618509 , 1.2798402, 0.8283719,
        1.2748733, 0.9506574, 1.3799721, 1.2126594, 1.0217606, 1.1820619,
        1.1487819, 1.243285 , 1.228684 , 1.0693493, 1.1452988, 1.4423909,
        1.0168364, 1.0897014, 1.2773035, 1.1495352, 1.326091 , 1.1542581,
        1.0180484, 1.2182463, 1.3834212, 1.1520929, 1.1436352, 1.2405603,
        1.2003627, 1.1742232, 1.2116325, 1.1945648, 1.2208644, 1.2573606],
      

In [26]:
kernel_shape = model.conv_layer.kernel_size
stride = model.conv_layer.stride[0]
print(f"Kernel Shape: {kernel_shape}\nStride: {stride}")

Kernel Shape: (5,)
Stride: 1


In [176]:
poly_mod_degree = 16384
bits_scale = 31
integer_scale = 40
coeff_mod_bit_sizes = [integer_scale, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, integer_scale]
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_eval.global_scale = 2 ** bits_scale
ctx_eval.generate_galois_keys()

In [177]:
enc_model = EncONEDCNN(model)
y_target_final = []
y_pred_final = []
one_part = y_test_20.shape[0] // 20
for i in range(1, 21):
    if i==1:
        j = one_part
        temp_x_test = x_test_20[:j,]
        temp_y_test = y_test_20[:j,]
    elif i==20:
        j = one_part * (i-1)
        temp_x_test = x_test_20[j:,]
        temp_y_test = y_test_20[j:,]
    else:
        j = one_part * i
        if i == 1:
            k = one_part
        else:
            k = one_part * (i-1)
        temp_x_test = x_test_20[k:j,]
        temp_y_test = y_test_20[k:j,]
    
    test_dataset = torch.utils.data.TensorDataset(temp_x_test, temp_y_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)
    
    t_start = time.time()
    enc_x_test = []
    print()
    print(f"Starting Batch {i} Encryption & Evaluation")
    target, output = enc_test(ctx_eval, enc_model, test_loader, criterion, kernel_shape, stride, tqdm_batch=i)
    t_end = time.time()
    print(f"Encryption & Evaluation of the test set took {int(t_end - t_start)} seconds")
    y_pred_final.extend(output)
    y_target_final.extend(target)


Starting Batch 1 Encryption & Evaluation


  0%|                                                                                         | 0/8514 [00:00<?, ?it/s]


ValueError: can only encrypt a vector

In [193]:
context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
plain_vector = [60, 66, 73, 81, 90]
encrypted_vector = ts.bfv_vector(context, plain_vector)
print("We just encrypted our plaintext vector of size:", encrypted_vector.size())
encrypted_vector

We just encrypted our plaintext vector of size: 5


In [194]:
encrypted_vector[1]

TypeError: 'BFVVector' object is not subscriptable